In [1]:
import os
os.chdir('..')

In [2]:
%pwd

'/home/ubuntu/Chicken-Disease-Classification'

In [3]:
# update the entity 


from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    """
    Configuration for data ingestion.
    """
    root_dir: Path 
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [4]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path= CONFIG_FILE_PATH,
            params_file_path= PARAMS_FILE_PATH
            ):


        print(f"Loading configuration from {config_file_path}")
        print(f"Loading parameters from {params_file_path}")

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])





    # data ingestion configuration 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.data_ingestion_root])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config.data_ingestion_root,
            source_url=config.source_url,
            local_data_file =config.local_data_file,
            unzip_dir =config.unzip_dir,
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
from ChickenDiseaseClassifier.utils.common import get_size


In [14]:
import shutil
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def copy_file(self):
        if not os.path.exists(self.config.local_data_file):
            # filename, headers = request.urlretrieve(
            #     url = self.config.source_URL,
            #     filename = self.config.local_data_file
            # )
            filename= self.config.local_data_file
            shutil.copy(self.config.source_url, filename)
            # logger.info(f"{filename} download! with following info: \n{headers}")
            logger.info(f"{filename} Copied into required destination \n")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(
        data_ingestion_config
    )
    data_ingestion.copy_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

Loading configuration from config/config.yaml
Loading parameters from params.yaml
[2023-08-17 02:55:52,692: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-17 02:55:52,693: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-17 02:55:52,694: INFO: common: created directory at: artifacts]
[2023-08-17 02:55:52,695: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-17 02:55:52,708: INFO: 3891229677: artifacts/data_ingestion/data.zip Copied into required destination 
]
